# การใช้งาน Semantic Kernel กับการเชื่อมต่อ OpenBnB MCP Server

โน้ตบุ๊กนี้แสดงวิธีการใช้ Semantic Kernel กับเซิร์ฟเวอร์ OpenBnB MCP จริงเพื่อค้นหาที่พัก Airbnb จริงโดยใช้ MCPStdioPlugin สำหรับการเข้าถึง LLM จะใช้ Azure AI Foundry ในการตั้งค่าตัวแปรสภาพแวดล้อมของคุณ คุณสามารถทำตาม [บทเรียนการตั้งค่า](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## การสร้างการเชื่อมต่อ MCP Plugin

เราจะเชื่อมต่อกับ [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) โดยใช้ MCPStdioPlugin เซิร์ฟเวอร์นี้ให้บริการฟังก์ชันการค้นหา Airbnb ผ่านแพ็กเกจ @openbnb/mcp-server-airbnb


## การสร้าง Client

ในตัวอย่างนี้ เราจะใช้ Azure AI Foundry สำหรับการเข้าถึง LLM ของเรา ตรวจสอบให้แน่ใจว่าคุณตั้งค่าตัวแปรสภาพแวดล้อมเรียบร้อยแล้ว


## การตั้งค่าสภาพแวดล้อม

กำหนดค่าการตั้งค่าของ Azure OpenAI ตรวจสอบให้แน่ใจว่าคุณได้ตั้งค่าตัวแปรสภาพแวดล้อมดังต่อไปนี้:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## การทำความเข้าใจการเชื่อมต่อ OpenBnB MCP

โน้ตบุ๊กนี้เชื่อมต่อกับ **เซิร์ฟเวอร์ OpenBnB MCP จริง** ที่ให้บริการฟังก์ชันการค้นหา Airbnb แบบจริง

### วิธีการทำงาน:

1. **MCPStdioPlugin**: ใช้การสื่อสารผ่านอินพุต/เอาต์พุตมาตรฐานกับเซิร์ฟเวอร์ MCP
2. **แพ็กเกจ NPM จริง**: ดาวน์โหลดและรัน `@openbnb/mcp-server-airbnb` ผ่าน npx
3. **ข้อมูลสด**: ส่งคืนข้อมูลอสังหาริมทรัพย์ Airbnb จริงจาก API ของพวกเขา
4. **การค้นพบฟังก์ชัน**: เอเจนต์ค้นพบฟังก์ชันที่มีอยู่จากเซิร์ฟเวอร์ MCP โดยอัตโนมัติ

### ฟังก์ชันที่มีอยู่:

เซิร์ฟเวอร์ OpenBnB MCP มักจะให้บริการ:
- **search_listings** - ค้นหาอสังหาริมทรัพย์ Airbnb ตามสถานที่และเกณฑ์
- **get_listing_details** - รับข้อมูลรายละเอียดเกี่ยวกับอสังหาริมทรัพย์เฉพาะ
- **check_availability** - ตรวจสอบความพร้อมใช้งานสำหรับวันที่เฉพาะเจาะจง
- **get_reviews** - ดึงรีวิวสำหรับอสังหาริมทรัพย์
- **get_host_info** - รับข้อมูลเกี่ยวกับเจ้าของอสังหาริมทรัพย์

### สิ่งที่ต้องเตรียม:

- **Node.js** ติดตั้งในระบบของคุณ
- **การเชื่อมต่ออินเทอร์เน็ต** เพื่อดาวน์โหลดแพ็กเกจเซิร์ฟเวอร์ MCP
- **NPX** พร้อมใช้งาน (มาพร้อมกับ Node.js)

### การทดสอบการเชื่อมต่อ:

คุณสามารถทดสอบเซิร์ฟเวอร์ MCP ด้วยตนเองโดยรัน:
```bash
npx -y @openbnb/mcp-server-airbnb
```

สิ่งนี้จะดาวน์โหลดและเริ่มต้นเซิร์ฟเวอร์ OpenBnB MCP ซึ่ง Semantic Kernel จะเชื่อมต่อเพื่อรับข้อมูล Airbnb จริง


## การใช้งาน Agent กับเซิร์ฟเวอร์ OpenBnB MCP

ตอนนี้เราจะใช้งาน AI Agent ที่เชื่อมต่อกับเซิร์ฟเวอร์ OpenBnB MCP เพื่อค้นหาที่พัก Airbnb จริงในสตอกโฮล์มสำหรับผู้ใหญ่ 2 คนและเด็ก 1 คน คุณสามารถเปลี่ยนรายการ `user_inputs` เพื่อปรับเปลี่ยนเกณฑ์การค้นหาได้ตามต้องการ


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# สรุป
ยินดีด้วย! คุณได้สร้าง AI agent ที่สามารถค้นหาที่พักในโลกจริงโดยใช้ Model Context Protocol (MCP) ได้สำเร็จ:

## เทคโนโลยีที่ใช้:
- Semantic Kernel - สำหรับสร้าง intelligent agents ด้วย Azure OpenAI
- Azure AI Foundry - สำหรับความสามารถของ LLM และการตอบสนองในแชท
- MCP (Model Context Protocol) - สำหรับการรวมเครื่องมือแบบมาตรฐาน
- OpenBnB MCP Server - สำหรับการค้นหาที่พัก Airbnb จริง
- Node.js/NPX - สำหรับการรัน MCP server ภายนอก

## สิ่งที่คุณได้เรียนรู้:
- การรวม MCP: การเชื่อมต่อ Semantic Kernel agents กับ MCP servers ภายนอก
- การเข้าถึงข้อมูลแบบเรียลไทม์: การค้นหาที่พัก Airbnb จริงผ่าน API แบบสด
- การสื่อสารผ่านโปรโตคอล: การใช้ stdio ในการสื่อสารระหว่าง agent และ MCP server
- การค้นพบฟังก์ชัน: การค้นหาฟังก์ชันที่มีอยู่จาก MCP servers โดยอัตโนมัติ
- การตอบสนองแบบสตรีม: การจับและบันทึกการเรียกฟังก์ชันแบบเรียลไทม์
- การแสดงผล HTML: การจัดรูปแบบการตอบสนองของ agent ด้วยตารางที่มีสไตล์และการแสดงผลแบบโต้ตอบ

## ขั้นตอนถัดไป:
- รวม MCP servers เพิ่มเติม (เช่น สภาพอากาศ, เที่ยวบิน, ร้านอาหาร)
- สร้างระบบ multi-agent ที่รวม MCP และ A2A protocols
- สร้าง MCP servers แบบกำหนดเองสำหรับแหล่งข้อมูลของคุณเอง
- ใช้หน่วยความจำการสนทนาแบบถาวรระหว่าง sessions
- นำ agent ไปใช้งานบน Azure Functions พร้อมการจัดการ MCP server
- เพิ่มความสามารถในการยืนยันตัวตนผู้ใช้และการจอง



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ แนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
